In [2]:
import os
import cv2
import numpy as np
from skimage.io import imshow

In [15]:
objects = ["arduino", "netcard", "drawcore"]
source_folder = (
    "/media/egor/T7/vut_dp_project_workspace/assets/datasets/custom_structured/"
)

mask_folder = "/media/egor/T7/vut_dp_project_workspace/source/playground/inspection/support/exports/_masks/"
original_folder = "/media/egor/T7/vut_dp_project_workspace/source/playground/inspection/support/exports/_originals/"
export_folder = "/media/egor/T7/vut_dp_project_workspace/source/playground/inspection/support/exports/"

extract_colors = [
    [np.array([255, 255, 0], np.uint8), "cap_polar"],  # RGB, cap. (polar)
    [np.array([255, 0, 0], np.uint8), "resister"],  # RGB, resister
    [np.array([0, 255, 0], np.uint8), "cap_nonpolar"],  # RGB, cap.
    [np.array([0, 0, 255], np.uint8), "missing"],  # RGB, miss
    [np.array([60, 120, 120], np.uint8), "led"],  # RGB, led
    [np.array([100, 180, 120], np.uint8), "ic"],  # RGB, IC
]

In [4]:
def align(im, template):
    # use ORB to detect keypoints and extract (binary) local
    # invariant features
    orb = cv2.ORB_create(1000)

    (kpsA, descsA) = orb.detectAndCompute(im, None)
    (kpsB, descsB) = orb.detectAndCompute(template, None)

    # match the features
    method = cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
    matcher = cv2.DescriptorMatcher_create(method)
    matches = matcher.match(descsA, descsB, None)

    # sort the matches by their distance (the smaller the distance,
    # the "more similar" the features are)
    matches = sorted(matches, key=lambda x: x.distance)
    # keep only the top matches
    keep = int(len(matches) * 0.5)
    matches = matches[:keep]

    # check to see if we should visualize the matched keypoints
    # matchedVis = cv2.drawMatches(im, kpsA, template, kpsB, matches, None)
    # matchedVis = imutils.resize(matchedVis, width=1000)
    # imshow(matchedVis)

    # allocate memory for the keypoints (x, y)-coordinates from the
    # top matches -- we'll use these coordinates to compute our
    # homography matrix
    ptsA = np.zeros((len(matches), 2), dtype="float")
    ptsB = np.zeros((len(matches), 2), dtype="float")
    # loop over the top matches
    for i, m in enumerate(matches):
        # indicate that the two keypoints in the respective images
        # map to each other
        ptsA[i] = kpsA[m.queryIdx].pt
        ptsB[i] = kpsB[m.trainIdx].pt

    # compute the homography matrix between the two sets of matched
    # points
    (H, mask) = cv2.findHomography(ptsA, ptsB, method=cv2.RANSAC)
    # use the homography matrix to align the images
    (h, w) = template.shape[:2]
    im = cv2.warpPerspective(im, H, (w, h))
    # return the aligned image
    return im


def cut(im, value):
    t = value[cv2.CC_STAT_TOP]
    l = value[cv2.CC_STAT_LEFT]

    w = value[cv2.CC_STAT_WIDTH]
    h = value[cv2.CC_STAT_HEIGHT]

    return im[t: t + h, l: l + w]

In [16]:
def extract(im, color: list[int], masked: np.ndarray):
    mask = cv2.inRange(masked, color, color)
    analysis = cv2.connectedComponentsWithStats(mask, cv2.CV_32S)
    (_, _, values, _) = analysis

    chunks = []
    im = np.multiply(align(im, original), mask)
    for i in range(1, len(values)):
        chunks.append(255 - cut(im, values[i]))
    return chunks


try:
    os.mkdir(export_folder)
except Exception as e:
    pass


for obj in objects:
    masked = cv2.cvtColor(  # Image with mask
        cv2.imread(f"{mask_folder}/{obj}.png"),
        cv2.COLOR_BGR2RGB,
    )
    original = cv2.cvtColor(  # Source image used for mask
        cv2.imread(f"{original_folder}/{obj}.bmp"),
        cv2.COLOR_RGB2GRAY,
    )

    for fld in os.listdir(source_folder):
        if "um" in fld and obj in fld:
            print(f"processing: {fld}")
            for f in os.listdir(f"{source_folder}/{fld}"):
                im = cv2.cvtColor(  # Image to extract components
                    cv2.imread(f"{source_folder}/{fld}/{f}"),
                    cv2.COLOR_RGB2GRAY,
                )
                for color, lbl in extract_colors:
                    try:
                        os.mkdir(f"{export_folder}/{lbl}")
                    except Exception as e:
                        pass
                    chunks = extract(im, color, masked)
                    for i, chunk in enumerate(chunks):
                        cv2.imwrite(f"{export_folder}/{lbl}/{fld}__{f}__{i}.bmp", chunk)

processing: 27_arduino_um_16mm_f8_5299
processing: 28_arduino_um_16mm_f8_4750
processing: 29_arduino_um_16mm_f8_6023
processing: 21_netcard_um_16mm_f8_4320
processing: 22_netcard_um_16mm_f8_3101
processing: 23_netcard_um_16mm_f8_5014
processing: 24_drawcore_um_16mm_f8_4580
processing: 25_drawcore_um_16mm_f8_3110
processing: 26_drawcore_um_16mm_f8_5236
